# Język Python - Wykład 7

Różnice Python 2.x i 3.x

 - print vs print()
 - UTF i kodowanie
 - zip vs zip_longest
 - metaclasses
 - nowe i stare klasy
 - slowa kluczowe, True i False
 - biblioteki

In [ ]:
s = "ala ma kota"

In [ ]:
s[0]

In [ ]:
s[0] = 'b'

Python 3.x
  - Unicode domyślnie (w tym ASCII)
  - osobny typ dla danych binarnych
  
Python 2.x
  - ASCII domyślnie
  - dane binarnie domyślnie
  - osobny typ dla Unicode (spoza ASCII)

Mark Lutz - Learning Python (4th edition)

In [ ]:
print("żółw")

In [ ]:
# o typach danych

help(str)  # from 2.x str
help(bytes)  # from 2.x unicode
help(bytearray)

In [ ]:
import sys
print(sys.getdefaultencoding())

In [ ]:
ord('a')

In [ ]:
hex(97)

In [ ]:
chr(97)

In [ ]:
chr(0xC4)

In [ ]:
import codecs
import tempfile
import os.path

filename = os.path.join( tempfile.gettempdir() , "L6" )
with codecs.open(filename, mode='w', encoding='iso-8859-2') as f:
    f.write(u'żółw')
    
with codecs.open(filename, encoding='iso-8859-2') as f:
    string = f.readline()
    print(string, repr(string))

## Chevron print

In [ ]:
import sys

print('Error!', file=sys.stderr)
print('Not an error', file=sys.stdout)
print('Error!', file=sys.stderr)